In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida
from models.polishing import polishing
from models.filtering import filtering

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_nhanes()
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=.05)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (2000, 95)
33.65% missing data
Class distribution: (array([1., 2.]), array([1000, 1000]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': {
        'input_channels':1,
        'output_channels':1,
        'channels_skip':4,
        'down_layers':[(24, 7, 1),
                       (46, 9, 5),
                       (96, 11, 1),
                       (96, 13, 1)],
        'need_sigmoid':True,
        'need_bias':True,
        'pad':'zero',
        'downsample_mode':'stride',
        'upsample_mode':'nearest',
        'act_fun':'LeakyReLU',
        'need1x1_up':True
    },# or a list containing layers sizes
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 501, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 4.23 sec - Loss 0.060790 - ACC 60.35% - ACC Mean 60.35% - AUC 64.99% - AUC Mean 64.99% - Deter 000
Ite 00050 - 2.12 sec - Loss 0.012180 - ACC 61.30% - ACC Mean 61.70% - AUC 65.76% - AUC Mean 66.37% - Deter 009
Ite 00091 - 1.44 sec - Loss 0.007462 - ACC 61.45% - ACC Mean 61.74% - AUC 66.12% - AUC Mean 66.40% - Deter 050
Early stop ite 91, rollback to correction of ite 41, whith acc of 63.45% and auc of 68.77%
OUR METHOD RUN 1/10 - acc: 63.45% +- 0.0% - auc: 68.77% +- 0.0%
Ite 00000 - 1.71 sec - Loss 0.143298 - ACC 60.85% - ACC Mean 60.85% - AUC 66.52% - AUC Mean 66.52% - Deter 000
Ite 00050 - 1.34 sec - Loss 0.054432 - ACC 60.25% - ACC Mean 61.79% - AUC 65.44% - AUC Mean 66.76% - Deter 044
Ite 00056 - 1.26 sec - Loss 0.051195 - ACC 61.25% - ACC Mean 61.66% - AUC 65.66% - AUC Mean 66.52% - Deter 050
Early stop ite 56, rollback to correction of ite 6, whith acc of 63.55% and auc of 69.48%
OUR METHOD RUN 2/10 - acc: 63.55% +- 0.0% - auc: 69.48% +- 0.0%
Ite 00000 - 0.95 sec - Lo

## Run MICE

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 60.85% +- 0.0% - auc: 65.3% +- 0.0%


## Run SFIL

In [6]:
sfil_accs, sfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='standard', random_state=i)
    sfil_accs.append(acc.mean())
    sfil_aucs.append(auc.mean())
    print(f'SFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SFIL GLOBAL - acc: {round(np.mean(sfil_accs) * 100, 2)}% +- {round(np.std(sfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(sfil_aucs) * 100, 2)}% +- {round(np.std(sfil_aucs) * 100, 2)}%')

SFIL RUN 1/10 - acc: 62.2% +- 3.95% - auc: 67.06% +- 4.58%
SFIL RUN 2/10 - acc: 62.35% +- 3.96% - auc: 65.82% +- 3.64%
SFIL RUN 3/10 - acc: 61.55% +- 3.18% - auc: 66.98% +- 2.33%
SFIL RUN 4/10 - acc: 63.1% +- 1.61% - auc: 68.14% +- 3.12%
SFIL RUN 5/10 - acc: 61.25% +- 2.73% - auc: 65.49% +- 3.33%
SFIL RUN 6/10 - acc: 61.65% +- 0.66% - auc: 66.91% +- 1.44%
SFIL RUN 7/10 - acc: 61.5% +- 1.7% - auc: 66.87% +- 1.79%
SFIL RUN 8/10 - acc: 60.95% +- 2.5% - auc: 66.1% +- 2.77%
SFIL RUN 9/10 - acc: 62.75% +- 1.89% - auc: 67.54% +- 2.68%
SFIL RUN 10/10 - acc: 60.9% +- 1.53% - auc: 66.35% +- 1.8%
SFIL GLOBAL - acc: 61.82% +- 0.71% - auc: 66.73% +- 0.76%


## Run PFIL

In [7]:
pfil_accs, pfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='panda', random_state=i)
    pfil_accs.append(acc.mean())
    pfil_aucs.append(auc.mean())
    print(f'PFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PFIL GLOBAL - acc: {round(np.mean(pfil_accs) * 100, 2)}% +- {round(np.std(pfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(pfil_aucs) * 100, 2)}% +- {round(np.std(pfil_aucs) * 100, 2)}%')

PFIL RUN 1/10 - acc: 59.6% +- 2.65% - auc: 63.54% +- 3.04%
PFIL RUN 2/10 - acc: 63.45% +- 2.15% - auc: 67.22% +- 2.58%
PFIL RUN 3/10 - acc: 61.1% +- 1.79% - auc: 66.15% +- 1.73%
PFIL RUN 4/10 - acc: 60.7% +- 1.56% - auc: 65.15% +- 1.18%
PFIL RUN 5/10 - acc: 60.95% +- 2.19% - auc: 65.49% +- 1.62%
PFIL RUN 6/10 - acc: 60.7% +- 1.85% - auc: 64.8% +- 2.98%
PFIL RUN 7/10 - acc: 61.0% +- 2.53% - auc: 65.2% +- 2.14%
PFIL RUN 8/10 - acc: 61.85% +- 2.6% - auc: 66.44% +- 3.16%
PFIL RUN 9/10 - acc: 59.85% +- 1.59% - auc: 63.55% +- 1.61%
PFIL RUN 10/10 - acc: 62.75% +- 3.41% - auc: 67.25% +- 3.77%
PFIL GLOBAL - acc: 61.2% +- 1.14% - auc: 65.48% +- 1.25%


## Run SPOL

In [8]:
spol_accs, spol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='standard', random_state=i)
    spol_accs.append(acc.mean())
    spol_aucs.append(auc.mean())
    print(f'SPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SPOL GLOBAL - acc: {round(np.mean(spol_accs) * 100, 2)}% +- {round(np.std(spol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(spol_aucs) * 100, 2)}% +- {round(np.std(spol_aucs) * 100, 2)}%')

SPOL RUN 1/10 - acc: 58.95% +- 3.62% - auc: 63.69% +- 3.83%
SPOL RUN 2/10 - acc: 61.0% +- 2.46% - auc: 65.7% +- 3.68%
SPOL RUN 3/10 - acc: 60.25% +- 2.16% - auc: 66.37% +- 2.37%
SPOL RUN 4/10 - acc: 61.25% +- 1.49% - auc: 65.83% +- 2.4%
SPOL RUN 5/10 - acc: 59.55% +- 1.43% - auc: 63.5% +- 1.0%
SPOL RUN 6/10 - acc: 60.6% +- 2.49% - auc: 65.49% +- 2.26%
SPOL RUN 7/10 - acc: 61.35% +- 2.01% - auc: 65.23% +- 2.95%
SPOL RUN 8/10 - acc: 61.15% +- 0.87% - auc: 65.38% +- 1.47%
SPOL RUN 9/10 - acc: 62.05% +- 0.56% - auc: 65.9% +- 1.67%
SPOL RUN 10/10 - acc: 61.65% +- 1.03% - auc: 65.19% +- 1.67%
SPOL GLOBAL - acc: 60.78% +- 0.91% - auc: 65.23% +- 0.88%


## Run PPOL

In [9]:
ppol_accs, ppol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='panda', random_state=i)
    ppol_accs.append(acc.mean())
    ppol_aucs.append(auc.mean())
    print(f'PPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PPOL GLOBAL - acc: {round(np.mean(ppol_accs) * 100, 2)}% +- {round(np.std(ppol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(ppol_aucs) * 100, 2)}% +- {round(np.std(ppol_aucs) * 100, 2)}%')

PPOL RUN 1/10 - acc: 59.7% +- 1.4% - auc: 64.07% +- 2.38%
PPOL RUN 2/10 - acc: 62.2% +- 1.78% - auc: 66.37% +- 1.53%
PPOL RUN 3/10 - acc: 62.2% +- 2.81% - auc: 66.55% +- 2.06%
PPOL RUN 4/10 - acc: 60.95% +- 2.28% - auc: 66.14% +- 1.69%
PPOL RUN 5/10 - acc: 59.75% +- 2.19% - auc: 64.53% +- 2.84%
PPOL RUN 6/10 - acc: 61.9% +- 1.6% - auc: 66.73% +- 1.82%
PPOL RUN 7/10 - acc: 61.0% +- 0.84% - auc: 65.79% +- 1.66%
PPOL RUN 8/10 - acc: 61.75% +- 2.68% - auc: 66.02% +- 3.34%
PPOL RUN 9/10 - acc: 63.05% +- 0.91% - auc: 67.28% +- 1.67%
PPOL RUN 10/10 - acc: 59.55% +- 1.96% - auc: 64.54% +- 2.08%
PPOL GLOBAL - acc: 61.2% +- 1.16% - auc: 65.8% +- 1.02%


## Run T-tests

In [10]:
for model, metrics in {
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'SFIL': {'ACC': sfil_accs, 'AUC': sfil_aucs},
        'PFIL': {'ACC': pfil_accs, 'AUC': pfil_aucs},
        'SPOL': {'ACC': spol_accs, 'AUC': spol_aucs},
        'PPOL': {'ACC': ppol_accs, 'AUC': ppol_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly better than MICE
Metric AUC - OUR METHOD is significantly better than MICE
Metric ACC - OUR METHOD is significantly better than SFIL
Metric AUC - OUR METHOD is significantly better than SFIL
Metric ACC - OUR METHOD is significantly better than PFIL
Metric AUC - OUR METHOD is significantly better than PFIL
Metric ACC - OUR METHOD is significantly better than SPOL
Metric AUC - OUR METHOD is significantly better than SPOL
Metric ACC - OUR METHOD is significantly better than PPOL
Metric AUC - OUR METHOD is significantly better than PPOL
